<a href="https://colab.research.google.com/github/NicoPatalagua/BigData/blob/master/Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto 
## Procesamiento distribuido con Pyspark

### Paula Oviedo - Nicolás Patalagua

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

from google.colab import drive
drive.mount('/content/gdrive')
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("app")
sc = SparkContext.getOrCreate();

     |████████████████████████████████| 217.8MB 57kB/s 
     |████████████████████████████████| 204kB 54.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=b1fd45cfae916c5a17ca32378393e38afde5d1fe7293315fdb5c5a221e92ef4c
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!wget https://bigdata20192.s3.amazonaws.com/uk-housing-prices-paid.zip

--2020-04-03 00:22:48--  https://bigdata20192.s3.amazonaws.com/uk-housing-prices-paid.zip
Resolving bigdata20192.s3.amazonaws.com (bigdata20192.s3.amazonaws.com)... 52.216.24.60
Connecting to bigdata20192.s3.amazonaws.com (bigdata20192.s3.amazonaws.com)|52.216.24.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 796204510 (759M) [application/zip]
Saving to: ‘uk-housing-prices-paid.zip’

uk-housing-prices-p 100%[===================>] 759.32M  59.7MB/s    in 11s     

2020-04-03 00:22:59 (69.7 MB/s) - ‘uk-housing-prices-paid.zip’ saved [796204510/796204510]



In [0]:
!unzip uk-housing-prices-paid.zip

Archive:  uk-housing-prices-paid.zip
replace price_paid_records.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: price_paid_records.csv  


In [0]:
path= "price_paid_records.csv"
ObjDt= sc.textFile(path)
ObjDt.take(20)

#### 1. Número de casas vendidas por año:

In [0]:
def Mapper1():
  for i in sys.stdin:
    ObjDt=re.split(",",i)
    ObjDate=str(ObjDt[2]).split("-")
    if len(ObjDt)!=0:
      print(ObjDate[0]+"\t"+"1")

In [0]:
def Reduce1():
  ObjCont=0
  ObjLast=""
  for j in sys.stdin:
    ObjDt=i.split()
    if ObjDt[0]!="Date of Transfer":
      ObjCont=ObjCont+int(ObjDt[1])
      ObjLast=ObjDt[0]
    else:
      print(ObjLast+"\t"+str(ObjCont))
      ObjCont=0
      ObjLast=ObjDt[0]
  print(ObjLast+"\t"+str(ObjCont))

#### 2. Precio promedio de las casas en cada ciudad:

In [0]:
def mapper2(i):
    ObjDt=i.split(",")
    ObjCity=ObjDt[6]
    ObjPrice=ObjDt[1]
    return (ObjCity,(float(ObjPrice),float(1)))

In [0]:
ObjMapper=ObjDt.map(lambda Obj: mapper2(Obj))
ObjMapper.take(7)
ObjMapper=ObjMapper.reduceByKey(lambda i,j:(i[0]+j[0],i[1]+j[1]))
ObjMapper.take(7)

In [0]:
ObjMean=ObjMapper.map(lambda i:(i[0],i[1][0]/i[1][1]))
ObjMean.take(10)
ObjMean=ObjMean.sortByKey()
ObjMean.take(100)

In [0]:
ObjMean.saveAsTextFile("s3://nicopatalagua/AnswerMean2")

#### 3. Casa con menor precio por ciudad:

#### 4. Listado de precios de menor a mayor en la ciudad de STAMFORD para 2015:

In [0]:
def mapper4(i):
    ObjPrice=ObjDt[1]
    ObjCity=ObjDt[6]
    ObjYear=ObjDt[2]
    ObjYear=ObjYear[0:4]
    if(ObjCity=="STAMFORD" and ObjYear=="2015"):
        return (ObjCity, int(ObjPrice))
    return "OtherCity"

In [0]:
ObjMapper=ObjDt.map(lambda Obj: mapper4(Obj))
ObjMapper.take(7)
ObjStamford=ObjMapper.filter(lambda Obj:"OtherCity" not in Obj)
ObjStamford.take(15)
ObjStamford1=ObjStamford.map(lambda Obj:(Obj[1],Obj[0]))
ObjStamford1.take(10)
ObjStamfordPrice=ObjStamford1.sortByKey()
ObjStamfordPrice.take(100)

In [0]:
ObjStanfordPrice.saveAsTextFile("s3://nicopatalagua/AnswerStamford5")

#### 5. Mes con mayor número de ventas en cada año:

#### 6. Conteo de condados  con el respectivo número de ciudades:

In [0]:
def mapper6(i):
    ObjDt=i.split(",")
    ObjCity=ObjDt[6]
    ObjCountry=ObjDt[8]
    return (ObjCountry,ObjCity)

In [0]:
ObjMapper=ObjDt.map(lambda Obj: mapper6(Obj))
ObjMapper.take(10)
ObjMapper1=ObjMapper.distinct()
ObjMapper1.take(10)

In [0]:
ObjCondado=ObjMapper1.map(lambda Obj:[Obj[1],1])
ObjCondado.take(10)
ObjCondado1=ObjCondado.reduceByKey(lambda i,j:i+j)
ObjCondado1.take(5)

In [0]:
ObjConteo=ObjCondado1.map(lambda Obj:[Obj[1],1])
ObjConteo.take(10)

ObjConteoAnswer=ObjConteo.sortByKey()
ObjConteoAnswer.take(100)

In [0]:
ObjConteoAnswer.saveAsTextFile("s3://nicopatalagua/Answer6")

In [0]:
def Mapper4(i):
  ObjIf= i.split(",")
  ObjYear= ObjIf[2].split("")[0].strip()
  ObjName= ObjIf[6].strip()
  if ObjYear == "2015" and ObjName == "STAMFORD":
      return True
ObjAnswer4=ObjDt.filter(Mapper4).map(lambda x:(float(x.split(",")[1]),1)).sortByKey()
ObjAnswer4.take(10)
ObjAnswer4.saveAsTextFile("s3://nicopatalagua/Answer4")